## Pandas

In [ ]:
%pip install polars
%pip install findspark 
%pip install pyarrow

In [1]:
from datetime import datetime
from glob import glob

from pandas import concat
from pandas import read_parquet

start = datetime.now()

#df = concat(map(read_parquet, glob("/workspaces/codespaces-jupyter/data/yellow_tripdata_202*-*.parquet")))
df1 = read_parquet("/workspaces/codespaces-jupyter/data/yellow_tripdata_2022-01.parquet")
df2 = read_parquet("/workspaces/codespaces-jupyter/data/yellow_tripdata_2022-02.parquet")
df3 = read_parquet("/workspaces/codespaces-jupyter/data/yellow_tripdata_2022-03.parquet")
#df4 = read_parquet("/workspaces/codespaces-jupyter/data/yellow_tripdata_2022-04.parquet")
df = concat([df1, df2, df3])

print(f"DF has {len(df)} rows.")

res = (
    df.groupby(["PULocationID", "DOLocationID"])
    .agg(
        {
            "total_amount": "sum",
            "fare_amount": "sum",
            "tolls_amount": "sum",
            "tip_amount": "sum",
            "congestion_surcharge": "sum",
            "trip_distance": "mean",
        }
    )
    .sort_values(by="fare_amount", ascending=False)
    .head(10)
)
print(res)
print(f"Runtime: {datetime.now() - start}")

DF has 9071244 rows.
                           total_amount  fare_amount  tolls_amount  \
PULocationID DOLocationID                                            
132          265              999767.75    843313.70      40627.84   
264          264              934169.06    694876.74      17985.22   
132          230              907339.59    661641.49      79032.16   
             48               716210.62    527388.55      61830.82   
107          140              485199.97    457340.21          6.55   
237          236              743588.54    416130.20         30.65   
236          237              669666.38    388055.32         26.21   
132          132              463789.81    379919.64       7459.92   
138          230              566962.72    368040.00      68523.69   
132          164              508147.13    365589.50      45369.58   

                           tip_amount  congestion_surcharge  trip_distance  
PULocationID DOLocationID                                    

## Polars

### Note: Restarted the kernel here.

In [1]:
import polars as pl
from datetime import datetime

start = datetime.now()

df = pl.read_parquet("/workspaces/codespaces-jupyter/data/yellow_tripdata_2022-0*.parquet")

print(f"DF has {len(df)} rows.")

res = (
    df.groupby(["DOLocationID","PULocationID"],False)
    .agg([
        pl.col("total_amount").sum(),
        pl.col("fare_amount").sum(),
        pl.col("tolls_amount").sum(),
        pl.col("tip_amount").sum(),
        pl.col("congestion_surcharge").sum(),
        pl.col("trip_distance").mean()
    ])
    .sort("fare_amount", reverse=True)
    .limit(10)
)
print(res)
print(f"Runtime: {datetime.now() - start}")

DF has 9071244 rows.
shape: (10, 9)
┌────────────┬────────────┬─────────┬────────────┬─────┬────────────┬──────────┬────────────┬────────────┐
│ DOLocation ┆ PULocation ┆ literal ┆ total_amou ┆ ... ┆ tolls_amou ┆ tip_amou ┆ congestion ┆ trip_dista │
│ ID         ┆ ID         ┆ ---     ┆ nt         ┆     ┆ nt         ┆ nt       ┆ _surcharge ┆ nce        │
│ ---        ┆ ---        ┆ bool    ┆ ---        ┆     ┆ ---        ┆ ---      ┆ ---        ┆ ---        │
│ i64        ┆ i64        ┆         ┆ f64        ┆     ┆ f64        ┆ f64      ┆ f64        ┆ f64        │
╞════════════╪════════════╪═════════╪════════════╪═════╪════════════╪══════════╪════════════╪════════════╡
│ 265        ┆ 132        ┆ false   ┆ 923458.15  ┆ ... ┆ 37412.54   ┆ 84490.67 ┆ 702.5      ┆ 21.503692  │
│ 264        ┆ 264        ┆ false   ┆ 858368.73  ┆ ... ┆ 16668.64   ┆ 112291.2 ┆ 45352.5    ┆ 2.797454   │
│            ┆            ┆         ┆            ┆     ┆            ┆ 2        ┆            ┆            │
│

## PySpark Pandas

### Note: Restarted Kernel here.

In [2]:
from datetime import datetime

from pyspark.pandas import read_parquet

start = datetime.now()

df = read_parquet("/workspaces/codespaces-jupyter/data/yellow_tripdata_2022-0*.parquet")

print(f"DF has {len(df)} rows.")

res = (
    df.groupby(["PULocationID", "DOLocationID"])
    .agg(
        {
            "total_amount": "sum",
            "fare_amount": "sum",
            "tolls_amount": "sum",
            "tip_amount": "sum",
            "congestion_surcharge": "sum",
            "trip_distance": "mean",
        }
    )
    .sort_values(by="fare_amount", ascending=False)
    .head(10)
)
print(res)
print(f"Runtime: {datetime.now() - start}")

/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_parquet`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


DF has 9071244 rows.


                           total_amount  fare_amount  tolls_amount  tip_amount  congestion_surcharge  trip_distance
PULocationID DOLocationID                                                                                          
132          265              999767.75    843313.70      40627.84    91104.56                 775.0      21.489116
264          264              934169.06    694876.74      17985.22   122516.65               49505.0       2.798411
132          230              907339.59    661641.49      79032.16    97780.44               31515.0      18.320027
             48               716210.62    527388.55      61830.82    73087.85               25145.0      19.112491
107          140              485199.97    457340.21          6.55    10980.61               11380.0      28.126166
237          236              743588.54    416130.20         30.65   102199.54              154310.0       1.090807
236          237              669666.38    388055.32         26.21    91